In [1]:
import random
import numpy as np
from typing import Callable
from gx_utils import *
import logging

N=7
SEED=42


In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [3]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))
        '''str_=''
        for list_ in self._data:
            #for el in list_:
            #    str_+=str(el)
            #str_+='|'
            str_+=str(list_)
        print(str_)
        if(str_==''):
            str_='[]'
        return hash(str_)'''
        

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()



In [4]:
P=np.array(problem(N,SEED),dtype=object)

INITIAL_STATE= State(np.array([],dtype=object))

In [5]:
def is_valid(frozenset_: np.array, action: list):
    #print(action)
    for list_ in frozenset_:
        #print(list_)
        if set(list_)==set(action):
            return False
    
    return True

#setEx=np.array([[0],[1,2],[3,4,5],[6]],dtype=object)
#actionEx=[2,1]

#print(is_valid(setEx,actionEx))


def possible_actions(state: State):
    return [list_ for list_ in P if is_valid(state._data, list_)]

#for a in possible_actions(State(setEx)):
#    print(a)

In [6]:
def result(state,action):
    temp=state.copy_data().tolist()
    #print(temp+[action])
    return State(np.array(temp+[action],dtype=object))

#setEx=np.array([[0],[1,2],[3,4,5],[6]],dtype=object)
#actionEx=[1,3]

#print(result(State(setEx),actionEx))

In [7]:
#GOAL = State(np.array(list(range(1, SIZE**2)) + [0]).reshape((SIZE, SIZE)))
#logging.info(f"Goal:\n{GOAL}")

'''
def goal_test(state):
    goal=np.zeros(N)
    for list_ in P:
        for el in list_:
            goal[el]=1
            if np.sum(goal)==N:
                return True
    if np.sum(goal)==N:
        return True
    else:
        return False            
'''


def goal_test(state):
    goal=set()
    for list_ in state._data:
        goal.update(list_)
    return len(goal)== N

#goal_test(P)


    


    

In [8]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for a in possible_actions(state):
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    print(s)
    while s:
        path.append(s.copy_data())
        s = parent_state[s]
    
    print(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))

BREADTH-FIRST

In [9]:
parent_state=dict()
state_cost=dict()

final=search(
    INITIAL_STATE,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: len(state_cost),
    unit_cost=lambda a: len(a),
)

[list([0]) list([5]) list([1, 3]) list([2, 4, 6])]
Found a solution in 5 steps; visited 193,923 states


A*

In [11]:
parent_state = dict()
state_cost = dict()


def h(state):
    goal=set()
    for list_ in state._data:
        goal.update(list_)
    g = set([i for i in range(N)])
    difference = g.difference(goal)
    return sum(difference)
    #return N-len(goal) 
    

#setEx=np.array([[0],[1,2],[3,4,5],[6]],dtype=object)
#actionEx=[2,1]

#print(set(setEx))
final = search(
    INITIAL_STATE,
    goal_test=goal_test,
    parent_state=parent_state,
    state_cost=state_cost,
    priority_function=lambda s: state_cost[s] + h(s),
    unit_cost=lambda a: len(a),
)

[list([2, 4, 6]) list([5]) list([1, 3]) list([0])]
Found a solution in 5 steps; visited 85 states
